## **障礙物觀點的 puzzle-based reshuffling**

### **集合**
---
$$
\begin{array}{ll}
   T & \text{時間集合，$T = \{1,2,3,...,|T|\}$}  \\
   N & \text{網格格點集合}  \\
   A & \text{格點間路徑集合}  \\
   R & \text{各類推盤集合，$R = \{0,1,2,...,|R|-1\}$}  \\
   \delta^{in}_{ri} & \text{推盤 $r\in R$ 流入格點 $i \in N$ 之節線集合， $\delta^{in}_{ri} \subset A$ } \\
   \delta^{out}_{ri} & \text{推盤 $r\in R$ 流出格點 $i \in N$ 之節線集合， $\delta^{out}_{ri} \subset A$ } \\
   V_i & \text{推盤垂直進出格點 $i \in N$ 之節線集合， $V_i \subset A$ } \\
   H_i & \text{推盤水平進出格點 $i \in N$ 之節線集合， $H_i \subset A$ } \\
   I & \text{推盤起始位置之集合，$I \subset N$} \\
   P & \text{推盤終點位置之集合，$P \subset N$} \\
\end{array}\\
$$

### **參數**
---

$$
\begin{array}{ll}
   M & \text{極大懲罰數} \\
   \epsilon & \text{極小的數} \\
   D & \text{虛擬迄點} \\
   S_i^r & \text{推盤 $r\in R$ 初始位置在格點 $i \in N$ 為1；反之為0} \\
   F_i^r & \text{推盤 $r\in R$ 終點位置在格點 $i \in N$ 為1；反之為0} \\
   Num^r & \text{各類推盤 $r \in R$ 數量} \\
   |T| & \text{揀貨時間上限} \\
   P_r & \text{推盤 $r\in R$ 的終點位置，$P_r \in N$} \\
\end{array}\\
$$

### **變數**
---
$$
\begin{array}{ll}
   f^{tr}_{ij} & \text{推盤 $r\in R$ 在時刻 $t \in T$ 自格點 $i \in N$ 移至格點 $j \in N$ 為1；反之為0}  \\
   x^t_r & \text{推盤 $r\in R$ 在時刻 $t \in T$ 抵達終點 $F_i^r$ 為1；反之為0} \\
   w & \text{出貨推盤盤送至揀貨點之最遲時刻，$w \in Integer^+$} \\
\end{array}\\
$$

In [2]:
from gurobipy import *

In [3]:
def print_dict(dict):
    print('{')
    for k, v in dict.items():
        print(f'\t{k}: {v}')
    print('}')
    
def print_2dlist(l):
    print('[')
    for row in l:
        print(f'\t{row}')
    print(']')

In [6]:
# import preprocess_obs
%run preprocess_obs.ipynb
########## 改版 ##########
# size = (4,4)
# n_sp = 5
# n_tar = 4
# id = 2
##### time limit #####
# size = (3,3)
# n_sp = 1
# n_tar = 8
# id = 2
size = (3,3)
n_sp = 2
n_tar = 3
id = 4
# map_grid, map_start, map_end, T_ub, T, N, R, A, In_ri, Out_ri, Vi, Hi, I, P, S_ri, F_ri, M, epsilon = preprocess_obs.preprocess(size, n_sp, n_tar, id)
map_grid, map_start, map_end, T_ub, T, N, R, A, In_ri, Out_ri, Vi, Hi, I, P, S_ri, F_ri, M, epsilon, Num = preprocess(size, n_sp, n_tar, id)

# ########## Sets ##########
# print(T)
# print(N)
# print(A)
# print(R)
# print_dict(Out_ri)
# print_dict(In_ri)
# print_dict(Vi)
# print_dict(Hi)
# print(I)
# print(P)

# ########## Parameters ##########
# print(M)
# print(epsilon)
# print_dict(S_ri)
# print_dict(F_ri)
print_2dlist(map_start)
print_2dlist(map_end)
print(Num)

[
	[2, -1, 0]
	[0, -1, -1]
	[-1, 1, 3]
]
[
	[3, -1, 2]
	[1, -1, -1]
	[-1, 0, 0]
]
[4, 1, 1, 1]


### **數學模式**
---
$$
\begin{array}{lll}
   Minimize & w + \epsilon *\displaystyle\sum_{r\in R} \displaystyle\sum_{t\in \{0\} \cup T} \displaystyle\sum_{(i,j) \in A, i \neq j} f^{tr}_{ij}\\
   \\
   Subject \quad To & \\
   \\
   \textbf{流量守恆}\\
   \\
   選一條出去(或沒有) & \displaystyle\sum_{(i,j) \in \delta^{out}_{ri}} f_{ij}^{0r} = S^r_i & \forall i \in N ; r \in R \\
   一條到終點(或沒有) & \displaystyle\sum_{(j,i) \in \delta^{in}_{ri}} f_{ji}^{|T|r} = F^r_i & \forall i \in N ; r \in R \backslash \{0\} \\
   各類推盤守恆 & \displaystyle\sum_{i \in N} \displaystyle\sum_{(j,i) \in \delta^{in}_{ri}} f_{ji}^{|T|r} = Num^r & \forall r \in R \\
   前時刻進=現時刻出 & \displaystyle\sum_{(j,i) \in \delta^{in}_{ri}} f_{ji}^{(t-1)r} = \displaystyle\sum_{(i,j) \in \delta^{out}_{ri}} f_{ij}^{tr} & \forall i \in N ; t \in T ; r \in R \\
    \\
    \textbf{碰撞限制} \\
    \\
    一點一托盤 & \displaystyle\sum_{r \in R} \displaystyle\sum_{(j,i) \in \delta^{in}_{ri}} f_{ji}^{tr} \leq 1 & \forall i \in N ; t \in \{0\} \cup T \\
    不可交換 & \displaystyle\sum_{r \in R} (f_{ji}^{tr} + f_{ij}^{tr}) \leq 1 & \forall i \in N ; (i,j) \in A,i \neq j ; t \in \{0\} \cup T \\
    水平進垂直出 & \displaystyle\sum_{r \in R} \displaystyle\sum_{(j_1,i) \in H_i} \displaystyle\sum_{(i,j_2) \in V_i} (f_{j_1i}^{tr} + f_{ij_2}^{tr}) \leq 1 & \forall i \in N ; t \in \{0\} \cup T \\
    垂直進水平出 & \displaystyle\sum_{r \in R} \displaystyle\sum_{(j_1,i) \in V_i} \displaystyle\sum_{(i,j_2) \in H_i} (f_{j_1i}^{tr} + f_{ij_2}^{tr}) \leq 1 & \forall i \in N ; t \in \{0\} \cup T \\
    \\
    \textbf{轉向限制} \\
    \\
    水平轉垂直 & \displaystyle\sum_{r \in R} \displaystyle\sum_{(j_1,i) \in H_i} \displaystyle\sum_{(i,j_2) \in V_i} (f_{j_1i}^{(t-1)r} + f_{ij_2}^{tr}) \leq 1 & \forall i \in N ; t \in T \\
    垂直轉水平 & \displaystyle\sum_{r \in R} \displaystyle\sum_{(j_1,i) \in V_i} \displaystyle\sum_{(i,j_2) \in H_i} (f_{j_1i}^{(t-1)r} + f_{ij_2}^{tr}) \leq 1 & \forall i \in N ; t \in T \\
    \\
    \textbf{時間限制} \\
    \\
    紀錄抵達終點時間 & \displaystyle\sum_{(P_r, j) \in A} f^{tr}_{P_rj} \geq x^t_r & \forall t \in \{0\} \cup T ; r \in R \backslash \{0\} \\
    至少經過終點一次 & \displaystyle\sum_{t \in \{0\} \cup T} x^t_r \geq 1 & \forall r \in R \backslash \{0\} \\
    最遲抵達終點時刻 & \displaystyle\sum_{t \in \{0\} \cup T} (t * x^t_r) \leq w & \forall r \in R \backslash \{0\} \\
    \\
    Bound & \\
          & f^{tr}_{ij} \in \{0,1\} & \forall r \in R ; (i,j) \in A ; t \in \{0\} \cup T \\
          & x^t_r \in \{0,1\} & \forall r \in R \backslash \{0\} ; t \in \{0\} \cup T\\
          & w \in Integer^+ \\
    \\
\end{array}\\            
$$

In [8]:
model = Model("Puzzle-Based Reshuffling")

f = {}
x = {}
w = 0
for t in [0]+T:
    for r in R:
        x[t,r] = model.addVar(vtype = "B", name = "x(%d,%d)" % (t,r))
        for i in N:
            for j in N:
                f[t,r,i,j] = model.addVar(vtype = "B", name = "f(%d,%d,%d,%d)" % (t,r,i,j))
                # x[t,r] = model.addVar(vtype = "B", name = "x(%d,%d)" % (t,r))

w = model.addVar(vtype = "I", name = "w")

for r in R:
    for i in N:
        # 時間 0，所有貨物 r 從起點出發遵守流量守恆
        model.addConstr(quicksum(f[0, r, i, j[1]] for j in Out_ri[i]) == S_ri[r][i], name = f"Start_R{r}_N{i}")
        # # 時間 T，所有貨物 r 抵達終點遵守流量守恆
        # if r != 0:
        #     model.addConstr(quicksum(f[T_ub, r, j[1], i] for j in In_ri[i]) == F_ri[r][i], name = f"End_R{r}_N{i}")
        # 所有相鄰時刻，同個點同個貨物要有一進一出
        for t in T:
            model.addConstr(quicksum(f[t-1, r, j[1], i] for j in In_ri[i]) == quicksum(f[t, r, i, j[1]] for j in Out_ri[i]),
                            name = f"Flow_balance_T{t}_R{r}_N{i}")
            
# 時間 T，所有推盤類別 r 抵達終點，且同類推盤總和遵守流量守恆
for r in R:
    model.addConstr(quicksum(f[T_ub, r, j[1], i] for i in N for j in In_ri[i]) == Num[r], name = f"End_R{r}")
            
for t in [0]+T:
    for i in N:
        # 一個格點只能有一個貨物
        model.addConstr(quicksum(quicksum(f[t, r, j[1], i] for j in In_ri[i]) for r in R) <= 1, 
                        name = f"Grid_{i}_Limit_in_time_{t}")
        # 禁止交換
        for j in A:
            if i != j[1]:
                model.addConstr(quicksum((f[t, r, j[1], i] + f[t, r, i, j[1]]) for r in R) <= 1, 
                                name = f"No_swapped_{i}_{j[1]}")
        # 同時刻的進出方向不能交錯
        model.addConstr(quicksum(f[t, r, j1[1], i] for j1 in Hi[i] for r in R) + 
                        quicksum(f[t, r, i, j2[1]] for j2 in Vi[i] for r in R) <= 1,
                        name = f"H{i}_in_V{i}_out")
        model.addConstr(quicksum(f[t, r, j1[1], i] for j1 in Vi[i] for r in R) + 
                        quicksum(f[t, r, i, j2[1]] for j2 in Hi[i] for r in R) <= 1,
                        name = f"V{i}_in_H{i}_out")
        
for t in T:
    for i in N:
        # 轉向限制停留 1 單位時間
        model.addConstr(quicksum(f[t-1, r, j1[1], i] for j1 in Hi[i] for r in R) + 
                        quicksum(f[t, r, i, j2[1]] for j2 in Vi[i] for r in R) <= 1,
                        name = f"H{i}_turn_V{i}")
        model.addConstr(quicksum(f[t-1, r, j1[1], i] for j1 in Vi[i] for r in R) + 
                        quicksum(f[t, r, i, j2[1]] for j2 in Hi[i] for r in R) <= 1,
                        name = f"V{i}_turn_H{i}")

# for r in R:
#     for t in [0]+T:
#         for (i,Pr) in A:
#             if Pr == P[r]:
#                 model.addConstr(w + M * (1 - f[t,r,i,Pr]) >= t)


print(f'target : {R[1:]}')
# 紀錄推盤編號及抵達時間
for t in [0]+T:
    for r in R[1:]:
        for endpos in P:
            if endpos[0] == r:
                model.addConstr(quicksum(f[t, r, endpos[1], j[1]] for j in Out_ri[endpos[1]]) >= x[t,r], name = f"Finish_T{t}_R{r}")
                
# 至少經過一次終點
for r in R[1:]:
    model.addConstr(quicksum(x[t,r] for t in [0]+T) >= 1,name = f"Finish_Position_R{r}")
# 最遲時刻
for r in R[1:]:
    model.addConstr(quicksum(t*x[t,r] for t in [0]+T) <= w, name = f"Time_Limit_R{r}")    

A_neq = []
for arc in A:
    if arc[0] != arc[1]:
        A_neq.append(arc)

is_running = False
timelimit = 350
model.Params.LogtoConsole = is_running  # 是否列出軟體求解過程
model.Params.timeLimit = timelimit      # 求解時限
model.setObjective(w + epsilon * quicksum(f[t,r,i,j] for t in [0]+T for r in R for (i,j) in A_neq) , GRB.MINIMIZE)
model.optimize()

def printModel(map_start):
    print("running time = ", model.Runtime)
    print("optimal value = ", model.objVal)
    map = []
    for row in map_start:
        map.append(row.copy())
    for t in [0]+T:
        print('-' * 40)
        print(f"time period {t}")
        for row in map:
            print('|', end = ' ')
            for col in row:
                if col == -1: col = ' X'
                print(f'{col:2}', end = ' ')
            print('|')

        map = []
        for i in range(size[0]):
            row = [0] * size[1]
            map.append(row)
        print()
        for r in R:
            for (i,j) in A:
                var_name = f"f({t},{r},{i},{j})"
                var = model.getVarByName(var_name)
                EPS = 1.e-6
                if var.X > EPS:
                    if i != j: 
                        print(var_name, var.X)
                    if r == 0:
                        map[(j-1)//size[0]][(j-1)%size[1]] = ' X'
                    else:
                        map[(j-1)//size[0]][(j-1)%size[1]] = r
                    

if model.status == GRB.OPTIMAL:
    model.write("model.lp")      # 可行約束條件
    # Print Optimal value and solution
    printModel(map_start)

elif model.status == GRB.TIME_LIMIT:
    print("Time out, cannot get a optimal solution")
    
elif model.status == GRB.INFEASIBLE:
    model.computeIIS()
    model.write("model.ilp")   # 寫出不可行的約束條件
    print("Infeasible, no solution found.")



target : [1, 2, 3]
running time =  13.197999954223633
optimal value =  8.016
----------------------------------------
time period 0
|  2  X  0 |
|  0  X  X |
|  X  1  3 |

f(0,0,2,3) 1.0
f(0,0,5,4) 1.0
f(0,2,1,2) 1.0
----------------------------------------
time period 1
|  0  2  X |
|  X  0  X |
|  X  1  3 |

f(1,1,8,5) 1.0
----------------------------------------
time period 2
|  0  2  X |
|  X  1  X |
|  X  0  3 |

f(2,0,4,1) 1.0
f(2,3,9,8) 1.0
----------------------------------------
time period 3
|  X  2  X |
|  0  1  X |
|  X  3  0 |

f(3,0,3,6) 1.0
f(3,0,6,9) 1.0
f(3,1,5,4) 1.0
----------------------------------------
time period 4
|  X  2  0 |
|  1  0  X |
|  X  3  X |

f(4,2,2,3) 1.0
f(4,3,8,5) 1.0
----------------------------------------
time period 5
|  X  0  2 |
|  1  3  X |
|  X  0  X |

f(5,0,7,8) 1.0
f(5,3,5,2) 1.0
----------------------------------------
time period 6
|  X  3  2 |
|  1  0  X |
|  0  X  X |

f(6,0,1,4) 1.0
f(6,1,4,7) 1.0
---------------------------------